In this iPython notebook, we **run logistic regression**. Code is largely copied from svm_logit.ipynb, but the output here is for data including an extra feature of tfbs.

In [21]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import cross_validation
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

<h3>Import data</h3>

In [22]:
CELLNAME = "GM12878"
label = "p300"

In [23]:
data = pd.read_table('data/' + CELLNAME + '_features.txt.gz', compression='gzip')
data.head(2)

,chr,lower,upper,H3K27ac,H3K27me3,H3K36me3,H3K4me1,H3K4me2,H3K4me3,H3K9ac,H4K20me1,p300,eRNA,tfbs
0,1,1,200,0,0,0,0,0,0,0,0,0,0,0
1,1,201,400,0,0,0,0,0,0,0,0,0,0,0


In [24]:
feature_names = list(data.columns.values)
feature_names = feature_names[3:]
feature_names.remove(label)
print feature_names

['H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me2', 'H3K4me3', 'H3K9ac', 'H4K20me1', 'eRNA', 'tfbs']


In [25]:
#Select positive set
positive = data[data[label]==1]
nrows = positive.shape[0]
print positive.shape

#Select negative set
use_random_sampling = False 
if use_random_sampling:
    negative = data[data[label]==0].as_matrix()
    nrows = negative.shape[0]
    select = np.random.choice(nrows, positive.shape[0]) # same size as positive set
    negative = negative[select,:]
    print negative.shape
else: # follow distribution of positive set
    indices = data[data[label]==1].index.values
    diff = np.diff(indices)
    offset = np.floor(np.mean(np.diff(indices))/2)
    select = (indices)+offset.astype(np.int64)
    mean_distance = np.mean(np.diff(indices))*200 # approximate value
    negative = data.iloc[select]
    negative = negative[negative[label]==0]
    print negative.shape

#Form sample
sample = pd.concat([positive,negative])
print sample.shape

(113883, 14)
(107085, 14)
(220968, 14)


In [35]:
X = sample.drop(['chr','lower','upper',label], axis=1).as_matrix()
add_interactions = True
if add_interactions:
    X = PolynomialFeatures(interaction_only=True).fit_transform(X) 
    X = X[:,1:]
    print X.shape
print X

(220968, 55)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [36]:
y = sample[label].as_matrix()
print y

[1 1 1 ..., 0 0 0]


In [37]:
print "Correlation between each feature and %s (Pearson corr coef)" %label
for i in range(len(feature_names)):
    print feature_names[i] + ": " + "%s" %stats.pearsonr(X[:,i], y)[0]

Correlation between each feature and p300 (Pearson corr coef)
H3K27ac: 0.348285510703
H3K27me3: 0.100417537633
H3K36me3: 0.0763629313715
H3K4me1: 0.332559484245
H3K4me2: 0.386937800594
H3K4me3: 0.354356924876
H3K9ac: 0.300208255893
H4K20me1: 0.118452444769
eRNA: 0.16743502311
tfbs: 0.0822085285811


<h3>Run Logistic Regression</h3>

In [38]:
penalty_type = 'l2'
regs = [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001]
for reg in regs: 
    logreg = linear_model.LogisticRegression(penalty=penalty_type, C=reg)
    logreg.fit(X,y)
    scores = cross_validation.cross_val_score(logreg, X, y, cv=10)
    mean_cv = sum(scores) / float(len(scores))
    print mean_cv

    coef = logreg.coef_
    coef = [item for sublist in coef for item in sublist] #flatten list
    coef_np = np.asarray(coef)
    top_coef = np.argsort(-coef_np)[:3]
    if add_interactions:
        print np.sort(-coef_np)

    #Plot
    if not add_interactions:
        ind = np.arange(len(feature_names))  
        width = 0.35       
        fig, ax = plt.subplots()
        rects = ax.bar(ind, coef, width, color='gray')
        for index in top_coef:
            rects[index].set_color('b')
        plt.xticks(range(len(feature_names)))
        ax.set_xticklabels(feature_names,rotation=45)
        plt.suptitle("Feature Coefficients (%s, C=%s)" %(penalty_type,reg))
        ax.annotate('CV Score = %0.4f'%mean_cv, xy=(0,1), xycoords='axes fraction', fontsize=12,
                        xytext=(5, -5), textcoords='offset points',
                        ha='left', va='top')
        plt.show()

0.713895449691
[19  5 23]
0.713913551901
[ 5  4 19]
0.713153278348
[5 4 3]
0.711139420276
[5 4 3]
0.705559390656
[4 3 5]
0.712411088368
[4 3 5]


Elastic Net (learned with Stochastic Gradient Descent) [unfinished work]

In [ ]:
from sklearn import linear_model
from sklearn import cross_validation
logregen = linear_model.SGDClassifier(loss='log', penalty='elasticnet', alpha=0.0001, l1_ratio=0.15)
logregen.fit(X,y)
print cross_validation.cross_val_score(logregen, X, y, cv=10)